<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/gpt4_1_fewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

client = OpenAI(api_key="")

In [ ]:
# =========================
# 1. МОДЕЛЬ
# =========================

MODEL_NAME = "gpt-4.1"

# =========================
# 2. FEW-SHOT (12 примеров)
# =========================

FEW_SHOT_BLOCK = """
Voici des exemples d’annotation selon le guide.

=== IDIOMES (opacité forte, noyau non littéral) ===
- lune de miel → expression_idiomatique
- forêt noire (gâteau) → expression_idiomatique
- peau de chagrin → expression_idiomatique

=== COLLOCATIONS OPAQUES (noyau littéral + métaphore/métonymie) ===
- fil rouge → collocation_opaque
- queue de poisson → collocation_opaque
- train de vie → collocation_opaque

=== COLLOCATIONS TRANSPARENTES (sens compositionnel) ===
- événement culturel → collocation_transparente
- roman policier → collocation_transparente
- bouteille en plastique → collocation_transparente

=== EXPRESSIONS LIBRES ===
- livre intéressant → expression_libre
- événement important → expression_libre
- maison ancienne → expression_libre

Rappels importants :
- Une expression idiomatique NE désigne PAS un type naturel du nom noyau.
- Une collocation opaque conserve un noyau nominal littéral.
- Si le noyau nominal reste interprétable littéralement → ce n’est PAS un idiome.
"""


# =========================
# 3. PROMPTS DES TESTS
# =========================

CRAN_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST CRAN — mot exclusivement figé
Question : l’expression contient-elle un mot exclusivement figé ?

Un mot exclusivement figé :
- n’existe pas comme mot autonome,
- n’apparaît jamais en dehors de l’expression,
- n’a plus d’usage productif moderne.

Exemple OUI : « us » dans « us et coutumes »
Exemple NON : « bleu » dans « cordon bleu »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MORPH_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MORPH — modification morphologique
Question : la flexion (pluriel, genre) rend-elle l’expression agrammaticale ou étrange ?

Exemple OUI : « garde du corps » → « gardes du corps »
Exemple NON : « livre scolaire » → « livres scolaires »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SYNT_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SYNT — modification syntaxique
Question : un changement d’ordre ou de structure modifie-t-il fortement le sens ou le rend-il agrammaticale ou bizzare ?

Exemple OUI : « pomme de terre » ≠ « pomme terrestre »
Exemple NON : « comité régional » ≈ « comité de la région »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

LEX_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST LEX — substitution lexicale
Question : remplacer un élément par un synonyme rend-il l’expression anormale, agrammaticale ou bizzare ?

Exemple OUI : « eau de vie » → « boisson de vie »
Exemple NON : « prix réduit » → « tarif réduit »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MODIF_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MODIF — insertion d’un modifieur
Question : l’insertion d’un modifieur (adjectif/adverbe) interne rend-elle l’expression anormale, agrammaticale ou bizzare ?

Exemple OUI : « pomme verte de terre »
Exemple NON : « roman contemporain policier »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SEMREST_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SEM-REST — restrictions sémantiques
Question : la tête nominale (le nom principal de l'expression) telle qu'elle est utilisée dans cette expression (dans son sens contextuel) a-t-elle un ensemble restreint de collocatifs naturels ?

Exemple OUI : « professeur de linguistique » → mais aussi « professeur de maths », « professeur de chimie », « professeur de littérature » (ensemble restreint aux domaines académiques)
Exemple NON : « événement majeur » → mais aussi « événement local », « événement imprévu », « événement politique », etc. (ensemble très ouvert)

⚠️ Attention : Tu dois analyser UNIQUEMENT la tête nominale, pas l’expression entière.
On vérifie avec quels collocatifs naturels se combine la tête nominale seule.

Exemples :
- dans « roman policier » → la tête nominale = « roman », pas « roman policier »
- dans « pomme de terre » → la tête nominale = « pomme », pas « pomme de terre »

Ne prends jamais l’expression complète comme base de comparaison.
Toujours la tête nominale seule.

⚠️ Attention : Dans ces données, il est rare que la réponse à ce test est NON.

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ID_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST ID — type naturel
Question : l’expression désigne-t-elle une sous-catégorie naturelle et littérale du noyau nominal ?


⚠️ Attention : tu dois identifier la tête syntaxique de l’expression :
le nom qui constitue le noyau de la construction nominale. L’objectif du test est de déterminer si l’expression désigne une sous-catégorie
naturelle de ce noyau.

Pour répondre OUI, le noyau nominal doit garder son sens et l’expression doit désigner un type de ce nom. La réponse est plus souvent OUI que NON, donc dans les cas d'incertitude penche vers OUI.

Exemples ID = NON :
- « île flottante » (dessert) : ce n’est pas un type d'île, c'est un dessert
- « forêt noire » (gâteau) : ce n’est pas un type de forêt, c'est un gâteau

Exemple ID = OUI :
- « événement culturel » : c’est un type d’événement
- « chaise en bois » : c’est un type de chaise
- « véhicule électrique » : c’est un type de véhicule
- « bouteille en plastique » : c’est un type de bouteille
- « porte coulissante » : c’est un type de porte



Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

OPAC_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

🔵 TEST OPAC — opacité sémantique

Question : le sens global ne correspond-il pas à la somme des sens littéraux des composants ?

Si le noyau nominal n’est pas littéral → OPAC = OUI.

Exemples OUI :
- « île flottante » (pas une île flotte)
- « forêt noire » (pas une forêt qui est noire)
- « lune de miel » (pas une lune construite de miel)

Exemples NON :
- « analyse des données » (c'est une analyse impliquant des données)
- « décision administrative » (c'est une décision qui est de type administratif)

Expression : « {expression} »
Contexte : {contexte}

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""


# =========================
# 4. APPEL D’UN TEST
# =========================

def ask_test(prompt_template, expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = (
        FEW_SHOT_BLOCK
        + "\n\n"
        + prompt_template.format(expression=expression, contexte=contexte)
    )

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=40,
    )

    full = (resp.choices[0].message.content or "").strip()
    first_line = full.split("\n")[0].strip().upper()
    decision = "OUI" if first_line == "OUI" else "NON"

    explanation = ""
    if "\n" in full:
        explanation = full.split("\n", 1)[1].strip()

    return decision, explanation


# =========================
# 5. ДЕРЕВО РЕШЕНИЙ
# =========================

def annotate_expression(expression, examples):
    tests = {}

    d, e = ask_test(CRAN_PROMPT, expression, examples)
    tests["CRAN"] = (d, e)
    if d == "OUI":
        return tests, "expression_idiomatique"

    d, e = ask_test(MORPH_PROMPT, expression, examples)
    tests["MORPH"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(SYNT_PROMPT, expression, examples)
    tests["SYNT"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(LEX_PROMPT, expression, examples)
    tests["LEX"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(MODIF_PROMPT, expression, examples)
    tests["MODIF"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(SEMREST_PROMPT, expression, examples)
    tests["SEM-REST"] = (d, e)
    if d == "NON":
        return tests, "expression_libre"

    d, e = ask_test(ID_PROMPT, expression, examples)
    tests["ID"] = (d, e)
    if d == "NON":
        return tests, "expression_idiomatique"

    d, e = ask_test(OPAC_PROMPT, expression, examples)
    tests["OPAC"] = (d, e)
    return tests, "collocation_opaque" if d == "OUI" else "collocation_transparente"


# =========================
# 6. FORMAT llm_raw_response
# =========================

def generate_annotation_with_llm(expression, examples):
    tests, category = annotate_expression(expression, examples)

    lines = []
    lines.append("Votes :")
    lines.append(f"- {category}")
    lines.append(f"\nCATÉGORIE FINALE : {category}")
    lines.append("\nExplications des tests menant à la catégorie finale :")

    for t in ["CRAN", "MORPH", "SYNT", "LEX", "MODIF", "SEM-REST", "ID", "OPAC"]:
        if t in tests:
            d, e = tests[t]
            lines.append(f"{t} : {d}")
            if e:
                lines.append(f"  Explication : {e}")

    return "\n".join(lines)


# =========================
# 7. TRAITEMENT EXCEL
# =========================

input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Annotation EP"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    df.at[idx, "llm_raw_response"] = generate_annotation_with_llm(expr, ex)

output_file = "annotated_nominal_part_7_gpt41_fewshot.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")


Annotation EP: 100%|██████████| 100/100 [10:30<00:00,  6.31s/it]

Saved: annotated_nominal_part_7_gpt41_fewshot.xlsx
